# LRU Cache Example
A record bin has a Map of Maps:  {“profile”:{“lut”:1714481010041, “meta”:{“provider”:“VI”}, “nlt”:1715085810041, “pct”:1714481010041, “status”:3}}

Need to build an SI on _status_ key's integer value (e.g. 3) and run an SI query on it.


### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed. 



In [55]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [47]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;


//Building map values
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.Value.MapValue;
import com.aerospike.client.Value;
import java.util.HashMap;
import java.util.Map;

//SI query related
import com.aerospike.client.policy.Policy;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.Statement;
import com.aerospike.client.cdt.CTX;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.query.IndexCollectionType;

//Batch read related
import com.aerospike.client.policy.BatchPolicy;
import com.aerospike.client.ResultCode;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);

In [88]:
%sh asadm --enable -e "manage sindex create numeric idx_mapval ns test set testset bin myMapBin1 ctx map_index(0) map_key(status) " -h "127.0.0.1"

In [89]:
//Add record data

//{“profile”:{“lut”:1714481010041, “meta”:{“provider”:“VI”}, “nlt”:1715085810041, “pct”:1714481010041, “status”:3}}

void addRecord(Integer keyIndex, String mapBinName, Integer statusVal, Long pctValue, Long nltValue, Long lutValue, String providerVal){
    MapPolicy mPolicy = new MapPolicy(MapOrder.UNORDERED, MapWriteFlags.DEFAULT);
    WritePolicy wPolicy = new WritePolicy();
    wPolicy.expiration = 10;
    wPolicy.sendKey = true;  //Optional, if you want to inspect the record key
    Key myRecKey = new Key("test", "testset", Value.get("key"+keyIndex));
    HashMap <String, Value> profileObj = new HashMap <String, Value>();
    profileObj.put("status", Value.get(statusVal));   
    profileObj.put("pct", Value.get(pctValue)); 
    profileObj.put("nlt", Value.get(nltValue)); 
    profileObj.put("lut", Value.get(lutValue)); 
    
    HashMap <String, String> metaObj = new HashMap <String, String>();
    metaObj.put("provider", providerVal);

    profileObj.put("meta", new MapValue(metaObj)); 

    client.operate(wPolicy, myRecKey, 
       MapOperation.put(mPolicy, mapBinName, Value.get("profile"), new MapValue(profileObj) )               
    );
    System.out.println("Rec added: "+ client.get(null, myRecKey));
}

//Add few records ... 
String binName = "myMapBin1";
addRecord(1, binName, 3, 1714481010041L, 1715085810041L, 1714481010041L, "VI3");
addRecord(2, binName, 4, 1714481010041L, 1715085810041L, 1714481010041L, "VI2");
addRecord(3, binName, 4, 1714481010041L, 1715085810041L, 1714481010041L, "VI4");
addRecord(4, binName, 6, 1714481010041L, 1715085810041L, 1714481010041L, "VI6");
addRecord(5, binName, 2, 1714481010041L, 1715085810041L, 1714481010041L, "VI2");
addRecord(6, binName, 3, 1714481010041L, 1715085810041L, 1714481010041L, "VI3");


Rec added: (gen:1),(exp:453091085),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI3}, nlt=1715085810041, status=3}}))
Rec added: (gen:1),(exp:453091085),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI2}, nlt=1715085810041, status=4}}))
Rec added: (gen:1),(exp:453091085),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI4}, nlt=1715085810041, status=4}}))
Rec added: (gen:1),(exp:453091085),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI6}, nlt=1715085810041, status=6}}))
Rec added: (gen:1),(exp:453091085),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI2}, nlt=1715085810041, status=2}}))
Rec added: (gen:1),(exp:453091085),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI3}, nlt=1715085810041, status=3}}))


In [84]:
//Run SI query

Filter filter = Filter.equal("myMapBin1", 3,
   CTX.mapIndex(0), CTX.mapKey(Value.get("status")));

//Note: Use same CTX construct as the SI declaration
// While CTX.mapKey("profile"), CTX.mapKey("status")
// will point to same value, it will result in sindex not found error.
QueryPolicy qPolicy = new QueryPolicy();
qPolicy.readTouchTtlPercent = 80;

Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(filter);
RecordSet rs = client.query(qPolicy, stmt);
while (rs.next()) {
   Key key = rs.getKey();
   Record record = rs.getRecord();
   System.out.format("key=%s bins=%s\n", key.userKey, record);
    //System.out.println(record);
  }
rs.close();


key=key1 bins=(gen:1),(exp:453091287),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI3}, nlt=1715085810041, status=3}}))
key=key6 bins=(gen:1),(exp:453091287),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI3}, nlt=1715085810041, status=3}}))


In [18]:
Policy rPolicy = new Policy();
rPolicy.readTouchTtlPercent = 80;
Key myKey1 = new Key("test", "testset", Value.get("key1"));
System.out.println("Rec read: "+ client.get(rPolicy, myKey1));
rPolicy.readTouchTtlPercent = 0;
Key myKey3 = new Key("test", "testset", Value.get("key3"));
System.out.println("Rec read: "+ client.get(rPolicy, myKey3));

Rec read: (gen:7),(exp:453089816),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI3}, nlt=1715085810041, status=3}}))
Rec read: (gen:7),(exp:453089816),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI4}, nlt=1715085810041, status=4}}))


In [85]:
//Do a batch read
BatchPolicy batchPolicy = new BatchPolicy();
batchPolicy.respondAllKeys = true;
batchPolicy.readTouchTtlPercent = 80;
Key key1 = new Key("test", "testset", Value.get("key1"));
Key key2 = new Key("test", "testset", Value.get("key2"));
Key key3 = new Key("test", "testset", Value.get("key3"));
Key key4 = new Key("test", "testset", Value.get("key4"));
Record [] bReads = client.get(batchPolicy, new Key[] {key1, key2, key3, key4});

// process the batch reads
for (int i = 0; i < bReads.length; i++) {
    Record bRead = bReads[i];    
    if (bRead != null) { //Check individual record
                 
        System.out.format("Result[%d]: ",i);
        System.out.println(bRead);
    }
    else {   //Expired records return null
        System.out.format("Record[%d] not found. \n", i);
    }
}


Result[0]: (gen:1),(exp:453091287),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI3}, nlt=1715085810041, status=3}}))
Result[1]: (gen:1),(exp:453091287),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI2}, nlt=1715085810041, status=4}}))
Result[2]: (gen:1),(exp:453091287),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI4}, nlt=1715085810041, status=4}}))
Result[3]: (gen:1),(exp:453091287),(bins:(myMapBin1:{profile={pct=1714481010041, lut=1714481010041, meta={provider=VI6}, nlt=1715085810041, status=6}}))


# Cleanup

In [90]:
%sh asadm --enable -e "manage sindex delete idx_mapval ns test set testset" -h "127.0.0.1"

In [91]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"